In [1]:
import pandas as pd
import copy
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.cluster import MiniBatchKMeans

from sklearn.metrics import recall_score, precision_score
#from sklearn.cross_validation import KFold, train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
%matplotlib inline

In [3]:
young = pd.read_csv('input/responses.csv')

In [8]:
young.head(10).columns[:19]

Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       'Punk', 'Hiphop, Rap', 'Reggae, Ska', 'Swing, Jazz', 'Rock n roll',
       'Alternative', 'Latino', 'Techno, Trance', 'Opera'],
      dtype='object')

In [25]:
young.iloc[:,:19].describe()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,Punk,"Hiphop, Rap","Reggae, Ska","Swing, Jazz",Rock n roll,Alternative,Latino,"Techno, Trance",Opera
count,1007.000000,1008.000000,1006.000000,1005.000000,1005.000000,1003.000000,1008.000000,1007.000000,1004.000000,1007.000000,1002.000000,1006.000000,1003.000000,1004.000000,1003.000000,1003.000000,1002.000000,1003.000000,1009.000000
mean,4.731877,3.328373,3.113320,2.288557,2.123383,2.956132,2.761905,3.471698,3.761952,2.361470,2.456088,2.910537,2.769691,2.759960,3.141575,2.828514,2.842315,2.338983,2.139742
std,0.664049,0.833931,1.170568,1.138916,1.076136,1.252570,1.260845,1.161400,1.184861,1.372995,1.301105,1.375677,1.214434,1.257936,1.237269,1.347173,1.327902,1.324099,1.184094
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,5.000000,3.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000
50%,5.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000
75%,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,5.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [47]:
young_music_full = young.dropna(axis=0)#.iloc[:,:19]

In [48]:
young_music_full

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow
5,5.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0,5.0,5.0,...,20.0,186.0,77.0,1.0,male,right handed,secondary school,no,city,block of flats
6,5.0,5.0,5.0,3.0,1.0,2.0,2.0,5.0,3.0,1.0,...,20.0,177.0,50.0,1.0,female,right handed,secondary school,no,village,house/bungalow
7,5.0,3.0,3.0,2.0,1.0,2.0,2.0,4.0,5.0,1.0,...,19.0,184.0,90.0,1.0,male,right handed,college/bachelor degree,no,city,house/bungalow
9,5.0,3.0,2.0,5.0,2.0,2.0,5.0,3.0,5.0,2.0,...,19.0,174.0,60.0,3.0,female,right handed,secondary school,no,city,block of flats
10,5.0,3.0,3.0,2.0,1.0,2.0,3.0,4.0,3.0,2.0,...,19.0,175.0,60.0,2.0,female,left handed,secondary school,no,city,block of flats
11,5.0,3.0,1.0,1.0,1.0,4.0,1.0,2.0,5.0,1.0,...,17.0,176.0,60.0,1.0,female,right handed,primary school,no,city,block of flats


In [49]:
young_music_full.shape#[:1]

(674, 150)

In [50]:
X=np.array(young_music_full.iloc[:,:19])

In [51]:
X.shape

(674, 19)

In [52]:
kmeans = MiniBatchKMeans(n_clusters=5,
...                          random_state=0,
...                          batch_size=6,
...                          max_iter=10).fit(X)

In [53]:
kmeans.cluster_centers_

array([[4.96153846, 3.01923077, 3.48076923, 3.59615385, 2.96153846,
        4.23076923, 4.05769231, 3.69230769, 3.92307692, 2.26923077,
        2.21153846, 3.17307692, 2.98076923, 4.11538462, 3.75      ,
        3.21153846, 3.96153846, 2.48076923, 3.51923077],
       [4.79411765, 3.22058824, 2.20588235, 2.86764706, 2.55882353,
        3.97058824, 3.35294118, 2.32352941, 4.70588235, 3.63235294,
        3.39705882, 1.73529412, 2.88235294, 3.30882353, 4.01470588,
        3.72058824, 2.60294118, 1.41176471, 3.22058824],
       [4.7625    , 3.5       , 3.2875    , 1.6125    , 1.425     ,
        1.85      , 1.725     , 3.775     , 2.7625    , 1.4       ,
        1.5875    , 3.2875    , 2.1       , 1.5125    , 1.9375    ,
        1.8875    , 1.975     , 2.4625    , 1.225     ],
       [4.69421488, 3.29752066, 3.52066116, 2.        , 1.97520661,
        2.90082645, 2.97520661, 3.80165289, 3.52066116, 1.80165289,
        2.07438017, 3.40495868, 3.01652893, 3.05785124, 2.96694215,
        2.578

In [55]:
predicted_music_group = kmeans.predict(X)

In [56]:
young_music_full['predicted_music_group']=predicted_music_group

/Users/thalia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
young_music_full#.shape

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats,predicted_music_group
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats,2
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats,4
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats,1
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow,3
5,5.0,3.0,2.0,3.0,2.0,3.0,3.0,2.0,5.0,5.0,...,186.0,77.0,1.0,male,right handed,secondary school,no,city,block of flats,1
6,5.0,5.0,5.0,3.0,1.0,2.0,2.0,5.0,3.0,1.0,...,177.0,50.0,1.0,female,right handed,secondary school,no,village,house/bungalow,2
7,5.0,3.0,3.0,2.0,1.0,2.0,2.0,4.0,5.0,1.0,...,184.0,90.0,1.0,male,right handed,college/bachelor degree,no,city,house/bungalow,2
9,5.0,3.0,2.0,5.0,2.0,2.0,5.0,3.0,5.0,2.0,...,174.0,60.0,3.0,female,right handed,secondary school,no,city,block of flats,0
10,5.0,3.0,3.0,2.0,1.0,2.0,3.0,4.0,3.0,2.0,...,175.0,60.0,2.0,female,left handed,secondary school,no,city,block of flats,3
11,5.0,3.0,1.0,1.0,1.0,4.0,1.0,2.0,5.0,1.0,...,176.0,60.0,1.0,female,right handed,primary school,no,city,block of flats,4


In [61]:
young_music_full.to_csv("predicted_music_group_df.csv")